In [0]:
#Import Dependencies 
import pandas as pd 
import csv 

In [67]:
#Load CSV 
path = "https://raw.githubusercontent.com/laurahallaman1/Pandas-Challenge/master/purchase_data.csv"

pymoli_data = pd.read_csv(path)
pymoli_data.head

<bound method NDFrame.head of      Purchase ID  ... Price
0              0  ...  3.53
1              1  ...  1.56
2              2  ...  4.88
3              3  ...  3.27
4              4  ...  1.44
..           ...  ...   ...
775          775  ...  3.54
776          776  ...  1.63
777          777  ...  3.46
778          778  ...  4.19
779          779  ...  4.60

[780 rows x 7 columns]>

In [68]:
# Player Count
#Total Number of Players

player_count = pymoli_data.loc[:, ["Gender", "SN", "Age"]]
player_count = player_count.drop_duplicates()
total_players = player_count.count()[0]
pd.DataFrame({"Total Players": [total_players]})




,Total Players
0,576


In [69]:
#Purchasing Analysis (Total)

#Number of Unique Items
#Average Purchase Price
#Total Number of Purchases
#Total Revenue

#Get statistics 
unique_items = len(pymoli_data["Item ID"].unique())
average_price = pymoli_data["Price"].mean()
purchase_count = pymoli_data["Price"].count()
total_purchase = pymoli_data["Price"].sum()



#Create Table 
data_table = pd.DataFrame({"Number of Unique Items": unique_items,
                              "Average Price": [average_price],
                              "Number of Purchases": [purchase_count],
                              "Total Revenue": [total_purchase]})

data_table

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,183,3.050987,780,2379.77


In [70]:
#Gender Demographics

#Percentage and Count of Male Players
#Percentage and Count of Female Players
#Percentage and Count of Other / Non-Disclosed

gender_count = player_count["Gender"].value_counts()
gender_percentage = gender_count / total_players
gender_demographics_final = pd.DataFrame({"Total Count": gender_count, "Percentage of Players": gender_percentage})


gender_demographics_final


,Total Count,Percentage of Players
Male,484,0.840278
Female,81,0.140625
Other / Non-Disclosed,11,0.019097


In [71]:
#Purchasing Analysis (Gender)
#The below each broken by gender

#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Average Purchase Total per Person by Gender

total_purchase = pymoli_data.groupby(["Gender"]).sum()["Price"].rename("Total Purchase Value")
purchase_average = pymoli_data.groupby(["Gender"]).mean()["Price"].rename("Average Purchase Price")
purchase_count = pymoli_data.groupby(["Gender"]).count()["Price"].rename("Purchase Count")

# Total Purchase PP
perperson_total = total_purchase / gender_demographics_final["Total Count"]

# Create DF 
purchase_data_by_gender = pd.DataFrame({"Purchase Count": gender_count, "Average Purchase Price": purchase_average, "Total Purchase Value": purchase_count, "Average Total Purchase Per Person": perperson_total})


# Display Table 
purchase_data_by_gender



,Purchase Count,Average Purchase Price,Total Purchase Value,Average Total Purchase Per Person
Female,81,3.203009,113,4.468395
Male,484,3.017853,652,4.065372
Other / Non-Disclosed,11,3.346000,15,4.562727


In [72]:
#Age Demographics

#The below each broken into bins of 4 years (i.e. <10, 10-14, 15-19, etc.)

# Making bins 
age_bins = [0, 9.90, 14.90, 19.90, 24.90, 29.90, 34.90, 39.90, 99999]
group_labels = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]

# Title bins
player_demographics["Age Ranges"] = pd.cut(player_demographics["Age"], age_bins, labels=group_labels)

# Calculate percents 
age_totals = player_demographics["Age Ranges"].value_counts()
age_percents = age_totals / total_players
age_demographics = pd.DataFrame({"Total Count": age_totals, "Percentage of Players": age_percents})

# Display Table 
age_demographics = age_demographics.sort_index()
age_demographics


,Total Count,Percentage of Players
<10,17,0.029514
10-14,22,0.038194
15-19,107,0.185764
20-24,258,0.447917
25-29,77,0.133681
30-34,52,0.090278
35-39,31,0.053819
40+,12,0.020833


In [73]:
#Purchase Count
#Average Purchase Price
#Total Purchase Value
#Average Purchase Total per Person by Age Group

# Making bins
pymoli_data["Age Ranges"] = pd.cut(purchase_data["Age"], age_bins, labels=group_labels)

# Title bins and make calculations 
purchase_total_byage = pymoli_data.groupby(["Age Ranges"]).sum()["Price"].rename("Total Purchase Value")
average_byage = pymoli_data.groupby(["Age Ranges"]).mean()["Price"].rename("Average Purchase Price")
purchases_byage = pymoli_data.groupby(["Age Ranges"]).count()["Price"].rename("Purchase Count")

# Calculate per person totals 
perperson_total_age = purchase_total_byage / age_demographics["Total Count"]

# Create DF 
purchase_data_by_age = pd.DataFrame({"Purchase Count": purchases_byage, "Average Purchase Price": average_byage, "Total Purchase Value": purchase_total_byage, "Average Purchase Per Person": perperson_total_age})

# Display Table 
purchase_data_by_age

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Per Person
Age Ranges,,,,
<10,23,3.353478,77.13,4.537059
10-14,28,2.956429,82.78,3.762727
15-19,136,3.035956,412.89,3.858785
20-24,365,3.052219,1114.06,4.318062
25-29,101,2.900990,293.00,3.805195
30-34,73,2.931507,214.00,4.115385
35-39,41,3.601707,147.67,4.763548
40+,13,2.941538,38.24,3.186667


In [74]:
#Top Spenders

#Identify the the top 5 spenders in the game by total purchase value, then list (in a table):

#SN
#Purchase Count
#Average Purchase Price
#Total Purchase Value

# Make Calculations
total_value = pymoli_data.groupby(["SN"]).sum()["Price"].rename("Total Purchase Value")
average = pymoli_data.groupby(["SN"]).mean()["Price"].rename("Average Purchase Price")
count = pymoli_data.groupby(["SN"]).count()["Price"].rename("Purchase Count")

# Convert to DataFrame
top_spenders = pd.DataFrame({"Total Purchase Value": total_value, "Average Purchase Price": average, "Purchase Count": count})

# Display Table
top_spenders_sorted = top_spenders.sort_values("Total Purchase Value", ascending=False)

# Convert to Percents and Fix Formatting 
top_spenders_sorted["Average Purchase Price"] = top_spenders_sorted["Average Purchase Price"].map("${:,.2f}".format)
top_spenders_sorted["Total Purchase Value"] = top_spenders_sorted["Total Purchase Value"].map("${:,.2f}".format)
top_spenders_sorted = top_spenders_sorted.loc[:,["Purchase Count", "Average Purchase Price", "Total Purchase Value"]]

# Display Table
top_spenders_sorted.head(10)







,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10
Ilarin91,3,$4.23,$12.70
Ialallo29,3,$3.95,$11.84
Tyidaim51,3,$3.94,$11.83
Lassilsala30,3,$3.84,$11.51


In [75]:
#Most Popular Items

#Identify the 5 most popular items by purchase count, then list (in a table):

#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value
# Extract item Data

popular_items_data = pymoli_data.loc[:,["Item ID", "Item Name", "Price"]]

# Calculations 
total_item_purchase = popular_items_data.groupby(["Item ID", "Item Name"]).sum()["Price"].rename("Total Purchase Value")
average_item_purchase = popular_items_data.groupby(["Item ID", "Item Name"]).mean()["Price"]
item_count = popular_items_data.groupby(["Item ID", "Item Name"]).count()["Price"].rename("Purchase Count")

# Create DF 
popular_items_data_pd = pd.DataFrame({"Total Purchase Value": total_item_purchase, "Item Price": average_item_purchase, "Purchase Count": item_count})

# Sort 
popular_items_data_count_sorted = popular_items_data_pd.sort_values("Purchase Count", ascending=False)

# Convert to percents and reformat 
popular_items_data_count_sorted["Item Price"] = popular_items_data_count_sorted["Item Price"].map("${:,.2f}".format)
popular_items_data_count_sorted["Purchase Count"] = popular_items_data_count_sorted["Purchase Count"].map("{:,}".format)
popular_items_data_count_sorted["Total Purchase Value"] = popular_items_data_count_sorted["Total Purchase Value"].map("${:,.2f}".format)
most_popular_items = popular_items_data_count_sorted.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

#Display DF 
most_popular_items.head(10)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
82,Nirvana,9,$4.90,$44.10
19,"Pursuit, Cudgel of Necromancy",8,$1.02,$8.16
103,Singed Scalpel,8,$4.35,$34.80
75,Brutality Ivory Warmace,8,$2.42,$19.36
72,Winter's Bite,8,$3.77,$30.16
60,Wolf,8,$3.54,$28.32


In [76]:
#Most Profitable Items

#Identify the 5 most profitable items by total purchase value, then list (in a table):

#Item ID
#Item Name
#Purchase Count
#Item Price
#Total Purchase Value

# Sort items by amount purchased 
profitable_items = popular_items_data_pd.sort_values("Total Purchase Value", ascending=False)

# Convert to percents and format table 
profitable_items["Purchase Count"] = profitable_items["Purchase Count"].map("{:,}".format)
profitable_items["Item Price"] = profitable_items["Item Price"].map("${:,.2f}".format)
profitable_items["Total Purchase Value"] = profitable_items["Total Purchase Value"].map("${:,.2f}".format)
most_profitable_items = profitable_items.loc[:,["Purchase Count", "Item Price", "Total Purchase Value"]]

#Display DF 
most_profitable_items.head(10)


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
82,Nirvana,9,$4.90,$44.10
145,Fiery Glass Crusader,9,$4.58,$41.22
92,Final Critic,8,$4.88,$39.04
103,Singed Scalpel,8,$4.35,$34.80
59,"Lightning, Etcher of the King",8,$4.23,$33.84
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77
78,"Glimmer, Ender of the Moon",7,$4.40,$30.80
72,Winter's Bite,8,$3.77,$30.16


In [0]:
#As final considerations:

#You must use the Pandas Library and the Jupyter Notebook.
#You must submit a link to your Jupyter Notebook with the viewable Data Frames.
#You must include a written description of three observable trends based on the data.
#See Example Solution for a reference on expected format.

#Three Observable Trends: 
#1. Male players vastly outnumber female and other/nonbinary players 
#2. Female and nonbinary customers purchase more and spend more per purchase than male 
#3. Many of the most profitable items are also the most popular ones, such as Oathbreaker, Nirvana, and Firey glass 
